In [1]:
# calculate tokens
token_c = next_prime(2^32)
token_m = next_prime(2^16)
tokens = [token_c * token_m ^ i for i in range(6)]
token_str = ",".join([str(token) for token in tokens])
print(f"{token_str=}")

token_str='4294967311,281479272661007,18447307092384415759,1208981164913597455597583,79232998604942436447498797071,5192693029572112457459728663642127'


In [2]:
from pwn import *

# find size of good subgroup
def get_subgroup(x):
    fs = [1]
    for d in range(1, 25):
        while True:
            res = ecm.one_curve(x // product(fs), factor_digits=d)
            if res[0] == 1:
                break
            fs.append(res[0])
    # throw away small factors, in case the generator is
    # already in some subgroup
    fs = [f for f in fs if f == 1 or (10^2 < f < 10^14)]
    return product(fs)

while True:
    # r = remote("34.124.157.94", int(19622))
    r = process(["python", "otp2.py"])
    # i've hardcoded a good prime for local testing, in reality this takes 50-ish connections
    
    r.recvuntil(b"p: ")
    P = int(r.recvline().strip().decode())
    subgroup = get_subgroup(P-1)
    # make do with 70 bits of subgroup, we can recover the rest by brute force
    if subgroup.nbits() < 70:
        r.close()
        continue

    r.recvuntil(b"pub: ")
    gs = safeeval.expr(r.recvline().strip().decode())

    r.sendline(token_str.encode())

    r.recvuntil(b"Token Hash: ")
    token_hash = int(r.recvline().strip().decode())

    r.recvuntil(b"OTP Hash: ")
    otp_hash = safeeval.expr(r.recvline().strip().decode())
    break

[x] Starting local process '/home/sy/miniconda3/envs/sage/bin/python'
[+] Starting local process '/home/sy/miniconda3/envs/sage/bin/python': pid 29808


In [3]:
# Math time!

Fp = Zmod(P)

g = Fp((5*7)^10)

gr = Fp(otp_hash[0])

grsk = gr ^ token_hash

grs_t = [Fp(x) for x in otp_hash[1:]]
grsk_tk = [x ^ token for x, token in zip(grs_t, tokens)]
grsk_tk_prod = product(grsk_tk)

gtk = grsk_tk_prod / grsk

In [4]:
from tqdm.auto import tqdm
from itertools import count

gc = g ^ token_c
restrict = (P-1) // subgroup
subgroup_dlog = discrete_log(gtk ^ restrict, gc ^ restrict, ord=subgroup)
print(f"dlog is {subgroup_dlog} mod {subgroup} ({subgroup.nbits()} bits)")

# if the subgroup is too small to recover all of the otp sum, we brute force for
# dlogs in the form of subgroup_dlog + n*subgroup
print(f"search up to subgroup_dlog+{2^97 // subgroup}*subgroup")
acc = gc ^ subgroup_dlog
dlog = subgroup_dlog
gcs = gc ^ subgroup
for _ in tqdm(count()):
    if acc == gtk:
        break
    acc *= gcs
    dlog += subgroup

print(f"dlog is {dlog} after verification")

dlog is 239805411936731889614526 mod 1389727973749439726529247 (81 bits)
search up to subgroup_dlog+114019*subgroup


0it [00:00, ?it/s]

dlog is 45909903418224678097783289171 after verification


In [5]:
# submit the token

otps = [(dlog // token_m ^ i) % token_m for i in range(6)]
otp_str = ",".join(str(otp) for otp in otps)
print(f"{otp_str=}")

r.clean()
r.sendline(otp_str.encode())

flag = r.recvuntil(b"}")
print(flag.decode())

otp_str='29255,16961,58364,58870,57989,37972'
Welcome Admin! grey{I_forgot_to_remove_the_debug_message_qgn9fe58yMwJdMRf}
